For more information about how to setup google sheets api:

Portuguese: https://developers.google.com/sheets/api/quickstart/python?hl=pt-br

English: https://developers.google.com/sheets/api/quickstart/python

``` python
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
```

In [88]:
import pandas as pd
from gsheets_utils import *

In [89]:
gs_client = GoogleSheetsClient(credentials_file='credentials.json')


# CONTROLE DE PROJETO

In [90]:
csc_df = gs_client.get_sheet_data_as_dataframe(
    spreadsheet_id='1Q-P0QxPWxH_rCLslEFetiht9-SmmGtwyd5TPsm2HAp0',
    range_name='Tarefas!A1:T1000'
)

In [121]:
csc_data = gs_client.get_sheet_data(
    spreadsheet_id='1Q-P0QxPWxH_rCLslEFetiht9-SmmGtwyd5TPsm2HAp0',
    range_name='Tarefas!A1:T1000'
)

In [92]:
pmo_data = gs_client.get_sheet_data(
    spreadsheet_id='1aD6NWbiaIS1H4N09nkimKGeKJeD9ux2coW9aY2ZM-rc',
    range_name='Tarefas!A1:T1000'
)

In [93]:
tlog_data = gs_client.get_sheet_data(
    spreadsheet_id='128DTCwkR6DcthbwQzGmlGfAUnv6KuNidlE4_RWhfiXY',
    range_name='Tarefas!A1:T3000'
)

In [94]:
engagement_survey_data = gs_client.get_sheet_data(
    spreadsheet_id='1Rxo642UduYjf1OvHmvgXS5UN4OBwXK1nfOOWXMcYBrA',
    range_name='Respostas ao formulário 1!A1:P100'
)

# Cleaning Data

In [125]:
import pandas as pd

def clean_and_transform_data(data):
    # Extrair colunas e dados
    columns = data[0]  # Primeira linha são os nomes das colunas
    data = data[1:]    # As linhas subsequentes são os dados

    # Corrigir linhas com menos colunas
    fixed_data = [row + [''] * (len(columns) - len(row)) for row in data]

    # Substituir caracteres especiais e espaços nos nomes das colunas
    clean_columns = [col.replace(" ", "_").replace("(", "").replace(")", "").replace("-", "_").replace("ç", "c").replace("ã", "a") for col in columns]

    # Criar o DataFrame do Pandas com os nomes de colunas limpos
    df = pd.DataFrame(fixed_data, columns=clean_columns)

    # Substituir strings vazias por None (valores nulos)
    df.replace('', None, inplace=True)
    
    # Converter colunas específicas para int, preenchendo valores inválidos com NaN
    df['Sprint'] = pd.to_numeric(df['Sprint'], errors='coerce')
    df['ID'] = pd.to_numeric(df['ID'], errors='coerce')

    df['Tempo_Efetivo_horas'] = df['Tempo_Efetivo_horas'].str.replace(',', '.').astype(float)
    df['Tempo_Estimado_horas'] = df['Tempo_Estimado_horas'].str.replace(',', '.').astype(float)

    # Converter colunas 'Inicio' e 'Conclusao' para datetime
    df['Inicio'] = pd.to_datetime(df['Inicio'], format='%d/%m/%Y', errors='coerce').dt.date
    df['Conclusao'] = pd.to_datetime(df['Conclusao'], format='%d/%m/%Y', errors='coerce').dt.date

    for column in df.columns:
        null_count = df[column].isnull().sum()
        print(f"Valores nulos em '{column}': {null_count}")
    
    return df

In [126]:
cleaned_csc_df = clean_and_transform_data(csc_data)

cleaned_csc_df.head()

Valores nulos em 'Sprint': 0
Valores nulos em 'ID': 444
Valores nulos em 'CTI': 437
Valores nulos em 'Classificacao': 826
Valores nulos em 'Task': 0
Valores nulos em 'Descricao': 646
Valores nulos em 'Entrega': 0
Valores nulos em 'Produto': 0
Valores nulos em 'Responsavel': 0
Valores nulos em 'Marcador': 826
Valores nulos em 'Funcao': 0
Valores nulos em 'Bitrix?': 0
Valores nulos em 'Status': 0
Valores nulos em 'Inicio': 0
Valores nulos em 'Conclusao': 0
Valores nulos em 'Observacao': 826
Valores nulos em 'Tempo_Estimado_horas': 0
Valores nulos em 'Tempo_Efetivo_horas': 0
Valores nulos em 'Tempo_Efetivo_Repasse_horas': 824
Valores nulos em 'Id_task_atrasada': 826


,Sprint,ID,CTI,Classificacao,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Funcao,Bitrix?,Status,Inicio,Conclusao,Observacao,Tempo_Estimado_horas,Tempo_Efetivo_horas,Tempo_Efetivo_Repasse_horas,Id_task_atrasada
0,1,80416.0,06.04.01-99,None,Edenred Serviços] Reuniões do sprint,None,Reuniões,Gestão,Viviane Pagnussat Cechetti,None,Data Analytics,Sim,Completed,2022-08-08,2022-08-19,None,12.0,13.33,None,None
1,1,80420.0,06.04.01-99,None,Edenred Serviços] Reuniões do sprint,None,Reuniões,Gestão,Ana Claudia Garcia,None,Produto,Sim,Completed,2022-08-08,2022-08-19,None,8.0,6.25,None,None
2,1,80422.0,06.04.03-01,None,Edenred Serviços] Gestão do projeto,None,Gestão do Projeto,Gestão,Bernardo Kuerten Dellagnelo,None,Projetos,Sim,Completed,2022-08-08,2022-08-19,None,12.0,13.50,None,None
3,1,80446.0,06.04.00-01,None,Edenred Serviços] Entendimento do ambiente Ed...,None,Entendimento dos dados,MIgração,Viviane Pagnussat Cechetti,None,Data Analytics,Sim,Completed,2022-08-08,2022-08-19,None,20.0,5.81,None,None
4,1,NaN,06.04.03-01,None,Ajuste de horas,None,Gestão do Projeto,Gestão,Ana Claudia Garcia,None,Produto,Sim,Completed,2022-08-08,2022-08-19,None,0.0,10.00,None,None


In [110]:
cleaned_tlog_df = clean_and_transform_data(tlog_data)

cleaned_tlog_df.head()

Valores nulos em 'Sprint': 5
Valores nulos em 'ID': 0
Valores nulos em 'CTI': 264
Valores nulos em 'Classificacao': 1076
Valores nulos em 'Task': 0
Valores nulos em 'Descricao': 892
Valores nulos em 'Entrega': 4
Valores nulos em 'Produto': 1
Valores nulos em 'Responsavel': 0
Valores nulos em 'Marcador': 1076
Valores nulos em 'Funcao': 0
Valores nulos em 'Bitrix?': 0
Valores nulos em 'Status': 0
Valores nulos em 'Inicio': 0
Valores nulos em 'Conclusao': 0
Valores nulos em 'Tempo_Estimado_horas': 2
Valores nulos em 'Tempo_Efetivo_horas': 0
Valores nulos em 'Tempo_Efetivo_Repasse_horas': 1076
Valores nulos em 'Id_task_atrasada': 1076
Valores nulos em 'Previsto_no_Planejamento?': 1076


,Sprint,ID,CTI,Classificacao,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Funcao,Bitrix?,Status,Inicio,Conclusao,Tempo_Estimado_horas,Tempo_Efetivo_horas,Tempo_Efetivo_Repasse_horas,Id_task_atrasada,Previsto_no_Planejamento?
0,1.0,71058,None,None,Suporte técnico,None,Supervisão Tecnica,QA,Vitor Paulon Avancini,None,Diretoria,Sim,Completed,2022-05-30,2022-06-10,20.0,0.00,None,None,None
1,1.0,70978,None,None,Gestão do projeto,None,Gestão do Projeto,Gestão,Bernardo Kuerten Dellagnelo,None,Projetos,Sim,Completed,2022-05-30,2022-06-10,20.0,22.75,None,None,None
2,1.0,70988,None,None,Acompanhamento do time e entendimento do negócio,None,Acompanhamento do time Edenred,QA,Vitor Gerber Weiss,None,Produto,Sim,Completed,2022-05-30,2022-06-10,64.0,27.00,None,None,None
3,1.0,70990,None,None,Acompanhamento do time e entendimento do negócio,None,Acompanhamento do time Edenred,QA,Carlos Alberto Juraszek Junior,None,Analytics Engineering,Sim,Completed,2022-05-30,2022-06-10,20.0,13.00,None,None,None
4,1.0,71060,None,None,Acompanhamento do time e entendimento do negócio,None,Acompanhamento do time Edenred,QA,Gabriel Lajús Maccarini,None,Analytics Engineering,Sim,Completed,2022-05-30,2022-06-10,48.0,8.96,None,None,None


In [111]:
cleaned_pmo_df = clean_and_transform_data(pmo_data)

cleaned_pmo_df.head()

Valores nulos em 'Sprint': 0
Valores nulos em 'ID': 0
Valores nulos em 'CTI': 2
Valores nulos em 'Classificacao': 213
Valores nulos em 'Task': 0
Valores nulos em 'Descricao': 125
Valores nulos em 'Entrega': 0
Valores nulos em 'Produto': 0
Valores nulos em 'Responsavel': 0
Valores nulos em 'Marcador': 213
Valores nulos em 'Funcao': 0
Valores nulos em 'Bitrix?': 0
Valores nulos em 'Status': 0
Valores nulos em 'Inicio': 0
Valores nulos em 'Conclusao': 0
Valores nulos em 'Tempo_Estimado_horas': 0
Valores nulos em 'Tempo_Efetivo_horas': 0
Valores nulos em 'Tempo_Efetivo_Repasse_horas': 212
Valores nulos em 'Id_task_atrasada': 211
Valores nulos em 'Previsto_no_Planejamento?': 213


,Sprint,ID,CTI,Classificacao,Task,Descricao,Entrega,Produto,Responsavel,Marcador,Funcao,Bitrix?,Status,Inicio,Conclusao,Tempo_Estimado_horas,Tempo_Efetivo_horas,Tempo_Efetivo_Repasse_horas,Id_task_atrasada,Previsto_no_Planejamento?
0,1,215208,06.02.02-00,None,Desenvolvimento,None,Desenvolvimento,PMO BI,Camila Bosa Custódio,None,Analytics Engineering,Sim,Completed,2023-11-13,2023-11-24,40.0,16.0,None,None,None
1,1,215426,06.05.04-00,None,Supervisão Técnica,None,Supervisão Tecnica,Gestão,Viviane Pagnussat Cechetti,None,Data Analytics,Sim,Completed,2023-11-13,2023-11-24,10.0,0.0,None,None,None
2,1,215210,06.04.01-99,None,Gestão do projeto,None,Gestão do Projeto,Gestão,Higino Neto,None,Projetos,Sim,Completed,2023-11-13,2023-11-24,10.0,20.0,None,None,None
3,2,226118,06.04.01-99,None,Gestão do projeto,None,Gestão do Projeto,Gestão,Higino Neto,None,Projetos,Sim,Completed,2023-11-27,2023-12-08,10.0,0.0,None,None,None
4,2,226120,06.02.02-00,None,Desenvolvimento,None,Desenvolvimento,PMO BI,Camila Bosa Custódio,None,Analytics Engineering,Sim,Completed,2023-11-27,2023-12-08,40.0,18.0,None,None,None


# SNOWFLAKE

Snowflake-Snowpark-Python compatibility issue. Need to downgrade snowflake-snowpark-python to 1.9.0 (for Python 3.11, 3.10, 3.9, 3.8) and install snowflake connector using below command,( as per https://docs.snowflake.com/en/developer-guide/python-connector/python-connector-pandas#installation)

´´´python
pip install snowflake-connector-python[pandas]
´´´

In [99]:
from config import *
from snowpark_utils import *
from snowflake.snowpark.session import Session
# from snowflake.connector.pandas_tools import write_pandas


In [100]:
session = ConnectSession()

Starting connection...
Connected to Schema: "SANDBOX"."DEV_IVON_GARCIA"


## Projects

1. Data Preprocessing:
* Fix Incomplete Rows: Corrects rows that have fewer columns than expected by padding them with empty strings.
* Clean Column Names: Replaces spaces and special characters in column names to ensure compatibility with Snowflake.

2. Creating the Pandas DataFrame: Uses the cleaned column names and fixed data.
    
3. Converting to Snowpark DataFrame: Converts the Pandas DataFrame to a Snowpark DataFrame.
    
4. Saving the DataFrame as a Table: Saves the DataFrame to Snowflake, using overwrite mode to replace any existing table with the same name.
    
5. Display Confirmation: Prints the first few rows of the newly created table to confirm successful creation.

In [101]:
def create_table(session, df, table_name):
    """
    Creates a table in Snowflake from a list of data. The first row of the data is used as column names.
    The table is created using Snowpark, and any existing table with the same name is overwritten.

    Parameters:
    session: Snowpark session object.
    data: List of lists containing the data. The first list should contain column names.
    table_name: The name of the table to be created in Snowflake.
    """
    print("Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:")
    print(df.dtypes)

    # Convert the Pandas DataFrame to a Snowpark DataFrame
    snowpark_df = session.create_dataframe(df)
    
    # Insert the data into Snowflake and overwrite any existing table with the same name
    snowpark_df.write.mode("overwrite").save_as_table(table_name)

    # Print a message indicating the table was created and show the first few rows of the table
    print('Table created:')
    df_table = session.table(table_name)
    df_table.show()

In [127]:
create_table(session, cleaned_csc_df, table_name='teste_csc')

Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:
Sprint                           int64
ID                             float64
CTI                             object
Classificacao                   object
Task                            object
Descricao                       object
Entrega                         object
Produto                         object
Responsavel                     object
Marcador                        object
Funcao                          object
Bitrix?                         object
Status                          object
Inicio                          object
Conclusao                       object
Observacao                      object
Tempo_Estimado_horas           float64
Tempo_Efetivo_horas            float64
Tempo_Efetivo_Repasse_horas     object
Id_task_atrasada                object
dtype: object
Table created:
--------------------------------------------------------------------------------------------------------------------------

In [113]:
create_table(session, cleaned_tlog_df, table_name='teste_tlog')

Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:
Sprint                         float64
ID                               int64
CTI                             object
Classificacao                   object
Task                            object
Descricao                       object
Entrega                         object
Produto                         object
Responsavel                     object
Marcador                        object
Funcao                          object
Bitrix?                         object
Status                          object
Inicio                          object
Conclusao                       object
Tempo_Estimado_horas           float64
Tempo_Efetivo_horas            float64
Tempo_Efetivo_Repasse_horas     object
Id_task_atrasada                object
Previsto_no_Planejamento?       object
dtype: object
Table created:
--------------------------------------------------------------------------------------------------------------------------

In [115]:
create_table(session, cleaned_pmo_df, table_name='teste_pmo')

Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:
Sprint                           int64
ID                               int64
CTI                             object
Classificacao                   object
Task                            object
Descricao                       object
Entrega                         object
Produto                         object
Responsavel                     object
Marcador                        object
Funcao                          object
Bitrix?                         object
Status                          object
Inicio                          object
Conclusao                       object
Tempo_Estimado_horas           float64
Tempo_Efetivo_horas            float64
Tempo_Efetivo_Repasse_horas     object
Id_task_atrasada                object
Previsto_no_Planejamento?       object
dtype: object
Table created:
--------------------------------------------------------------------------------------------------------------------------

In [105]:
create_table(session, engagement_survey_data, table_name='teste_survey')

Tipos de dados do DataFrame Pandas antes de enviar para o Snowflake:


AttributeError: 'list' object has no attribute 'dtypes'

# Feedback

In [18]:
create_table_sql = """
CREATE OR REPLACE TABLE feedback (
    timestamp STRING,
    email STRING,
    sprint_ref STRING,
    squad STRING,
    engagement INT,
    development INT,
    quality INT,
    on_time INT,
    satisfaction INT,
    comments STRING
)
"""

session.sql(create_table_sql).collect()

---------------------------------------------------------------------------------------------------------------------------------------
|"TIMESTAMP"  |"EMAIL"  |"SPRINT_REF"  |"SQUAD"  |"ENGAGEMENT"  |"DEVELOPMENT"  |"QUALITY"  |"ON_TIME"  |"SATISFACTION"  |"COMMENTS"  |
---------------------------------------------------------------------------------------------------------------------------------------
|             |         |              |         |              |               |           |           |                |            |
---------------------------------------------------------------------------------------------------------------------------------------



In [19]:
for row in sheet_data[1:]:
    session.sql(f"""
    INSERT INTO feedback (timestamp, email, sprint_ref, squad, engagement, development, quality, on_time, satisfaction, comments)
    VALUES ('{row[0]}', '{row[1]}', '{row[2]}', '{row[3]}', {row[4]}, {row[5]}, {row[6]}, {row[7]}, {row[8]}, '{row[9]}')
    """).collect()

In [20]:
# Inserir os dados
for row in sheet_data[1:]:
    values = "', '".join(row)  # Preparar os valores para inserção
    session.sql(f"""
    INSERT INTO feedback ({', '.join(column_names_sql)})
    VALUES ('{values}')
    """).collect()

NameError: name 'column_names_sql' is not defined

In [ ]:
customer_wrt = df_table.write.mode("overwrite").save_as_table("SANDBOX.DEV_IVON_GARCIA.feedback")

In [ ]:
column_names = sheet_data[0]
column_names

In [ ]:
column_names_sql = [name.replace(' ', '_').replace('(', '').replace(')', '').replace('\n', '') for name in column_names]


In [ ]:
column_names_sql

In [ ]:
# from snowflake import snowpark
# from config import *
# SNOWFLAKE_CONFIG = {
#         "account": account_id,
#         "user": username,
#         "password": password,
#         "role": role,
#         "warehouse": warehouse,
#         "database": database,
#         "schema": schema}
# connection = snowpark.Session(SNOWFLAKE_CONFIG)
# # write_pandas(conn=connection, df=sheet_df, table_name='teste_kpi')

In [ ]:
pandas_df

In [ ]:
snowpark_df = session.write_pandas(conn=session, df=pandas_df, table_name="asas", auto_create_table=True, table_type="temp")

In [ ]:
snowpark_df.sort('"city"').to_pandas()

In [ ]:
def write_dataframe(session, df, table_name):
        success, nchunks, nrows, _ = write_pandas(session, df, table_name)
        if success:
            print(f"Successfully inserted {nrows} rows into {table_name}")
        else:
            print("Failed to insert data")

# Escrever dados no Snowflake
if sheet_data is not None:
    # Certifique-se de que o DataFrame tem as colunas corretas e tipos de dados
    sheet_data.columns = sheet_data.iloc[0]
    # sheet_data = sheet_data.drop(0)
    
    write_dataframe(session, sheet_data, "YOUR_TABLE_NAME")
    session.close()
else:
    print("No data to write to Snowflake")